# Coursera Capstone Project

## The Battle of Neighborhoods (Week 1)

### Angeliki Kalamara - Part 1 : Introduction and Data Sections

In [1]:
import pandas as pd 
import numpy as np 
import time

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


# 1. Introduction - Business Problem

## Discussion of the business problem and the audience who would be interested in this project.
## Description of the Problem and Background
### Scenario:

I am a data scientist residing in Utrecht, Netherlands. I currently live within walking distance to the tram station I enjoy many ammenities and venues in the area, such as various international cousine restaurants, cafes, food shops and entertainment. I have been offered a great opportunity to work for a leader firm in Manhattan, NY. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen. The key question is : How much do I need to spend in comparison to Melbourne to find an apartment with my partner which is as convenient and affordable as mine in Utrecht, considering the salary difference. I can use available real estate websites and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:

1. Apartment must be 2 or 3 bedrooms
2. Desired location is near a metro station in the Manhattan area and within 1.0 mile (1.6 km) radius
3. Price of rent not exceed 7,000 dollars  per month
4. Top ammenities in the selected neighborhood shall be similar to current residence
5. Desirable to have venues such as coffee shops, restaurants Asian Thai, wine stores, gym and food shops
6. as a reference, I have included a map of venues near current residence in Utrecht.

### Business Problem:
The challenge is to find a suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

### Interested Audience
I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

# 2. Data Section

## Description of the data and its sources that will be used to solve the problem

### The following data is required to answer the issues of the problem:

* List of Boroughs and neighborhoods of Manhattan with their geodata (latitud and longitud)
* List of Subway metro stations in Manhattan with their address location
* List of apartments for rent in Manhattan area with their addresses and price
* Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
* Venues for each Manhattan neighborhood ( than can be clustered)
* Venues for subway metro stations, as needed

## How the data will be used to solve the problem
### The data will be used as follows:

* Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
* Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
* Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
* Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
* Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.

The procesing of these DATA will allow to answer the key questions to make a decision:

* What is the cost of rent (per square ft) around a mile radius from each subway metro station?
* What is the area of Manhattan with best rental pricing that meets criteria established?
* What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
* What are the venues of the two best places to live? How the prices compare?
* How venues distribute among Manhattan neighborhoods and around metro stations?
* Are there tradeoffs between size and price and location?
* Any other interesting statistical data findings of the real estate and overall data.

## Reference of venues around current residence in Utrecht for comparison to Manhattan place

In [2]:
# Van Heuven Goedhartlaan, Netherlands
address = 'Van Heuven Goedhartlaan, Utrecht'

#address = 'Riverside Quay, Southbank'

geolocator = Nominatim(user_agent='coursera-capstone-project')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Utrecht home are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Utrecht home are 52.0773974, 5.086263.


In [3]:
neighborhood_latitude=52.0773974
neighborhood_longitude=5.086263

In [4]:
# @hidden_cell
CLIENT_ID = '0TILJ2HJ02ECJRT1HKXLE2RGFJ2KZDGRTLD3LT2JQKSFYLPJ' # your Foursquare ID
CLIENT_SECRET = '30CLYLN2WX5PF3SDRTNJHRN33IUD1SCXRNUOG1SELYDFP51Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0TILJ2HJ02ECJRT1HKXLE2RGFJ2KZDGRTLD3LT2JQKSFYLPJ
CLIENT_SECRET:30CLYLN2WX5PF3SDRTNJHRN33IUD1SCXRNUOG1SELYDFP51Q


In [5]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=0TILJ2HJ02ECJRT1HKXLE2RGFJ2KZDGRTLD3LT2JQKSFYLPJ&client_secret=30CLYLN2WX5PF3SDRTNJHRN33IUD1SCXRNUOG1SELYDFP51Q&v=20180605&ll=52.0773974,5.086263&radius=500&limit=100'

In [6]:
results = requests.get(url).json()
#results

Function that extracts the category of the venues near Utrecht (as borrowed from the Foursquare lab)

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
    
UTnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
UTnearby_venues = UTnearby_venues.loc[:, filtered_columns]

# filter the category for each row
UTnearby_venues['venue.categories'] = UTnearby_venues.apply(get_category_type, axis=1)

# clean columns
UTnearby_venues.columns = [col.split(".")[-1] for col in UTnearby_venues.columns]

UTnearby_venues.head(10)

/Users/angeliki/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Tennisclub Domstad,Tennis Court,52.081005,5.082411
1,Aquacenter Den Hommel,Pool,52.080883,5.084522
2,Utrechtse Bazaar,Flea Market,52.077194,5.079984
3,Tramhalte 24 Oktoberplein Zuid,Light Rail Station,52.078889,5.092793



### Map of Utrecht with venues near residence place - for reference

In [9]:
# create map of Singapore place  using latitude and longitude values
map_ut = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(UTnearby_venues['lat'], UTnearby_venues['lng'], UTnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_ut)  
    
map_ut


## 3.	Methodology Section

(tba)

## 4. Results Section

(tba)

## 5. Discussion Section

(tba)

## 6. Conclusion Section
(tba)